In [0]:
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_train.csv
!wget https://raw.githubusercontent.com/AJV009/Machine_Learning_NB/master/datasets/doctor_data/doctor_test.csv
!pip uninstall -y mkl
!pip install --upgrade mxnet
!pip install autogluon
!pip install -U ipykernel
from IPython import display
display.clear_output()

In [1]:
import pandas as pd
df = pd.read_csv('doctor_train.csv')
# df[27900:]
from autogluon import TabularPrediction as task
import autogluon as ag
# train_data = task.Dataset(df)
train_data = task.Dataset(df[:27800])
val_data = task.Dataset(df[27800:])
label_column = 'Y'

/usr/local/lib/python3.6/dist-packages/mxnet/optimizer/optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [0]:
hp_tune = True
nn_options = { # specifies non-default hyperparameter values for neural network models
              # number of training epochs (controls training time of NN models)
              'num_epochs': ag.space.Categorical(20,10),
               # learning rate used in training (real-valued hyperparameter searched on log-scale)
              'learning_rate': ag.space.Real(1e-8, 1e-1, default=5e-4, log=True),
              # activation function used in NN (categorical hyperparameter, default = first entry)
              'activation': ag.space.Categorical('relu', 'softrelu', 'tanh'),
              # Each choice for categorical hyperparameter 'layers' corresponds to list of sizes for each NN layer to use
              'layers': ag.space.Categorical([100],[1000],[200,100],[2000,1000,500,50],[300,200,100,50]),
              # dropout probability (real-valued hyperparameter)
              'dropout_prob': ag.space.Real(0.0, 0.5, default=0.1),
              }

gbm_options = { # specifies non-default hyperparameter values for lightGBM gradient boosted trees
               # number of boosting rounds (controls training time of GBM models)
               'num_boost_round': ag.space.Categorical(500,200,100,50),
                # number of leaves in trees (integer hyperparameter)
               'num_leaves': ag.space.Int(lower=10, upper=100, default=36),
               }

# If one of these keys is missing from hyperparameters dict, then no models of that type are trained.
hyperparameters = {'NN': nn_options, 'GBM': gbm_options}  # hyperparameters of each model type

time_limits = 45*60
num_trials = 20  # try at most 3 different hyperparameter configurations for each type of model
search_strategy = 'skopt'  # to tune hyperparameters using SKopt Bayesian optimization routine
output_directory = 'agModels-predictOccupation'  # folder where to store trained models

In [0]:
predictor = task.fit(train_data=train_data, tuning_data=val_data, label=label_column,
                     output_directory=output_directory, time_limits=time_limits, num_trials=num_trials,
                     hyperparameter_tune=hp_tune, hyperparameters=hyperparameters,
                     search_strategy=search_strategy, )
print(predictor.fit_summary())

In [5]:
metric = 'balanced_accuracy'
predictor = task.fit(train_data=train_data, label=label_column, eval_metric=metric, auto_stack=True,
                     hyperparameters = hyperparameters, time_limits = time_limits)
print(predictor.fit_summary())

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200518_154237/
Beginning AutoGluon training ... Time limit = 2700s
AutoGluon will save models to AutogluonModels/ag-20200518_154237/
Train Data Rows:    27800
Train Data Columns: 18
Preprocessing data ...
Here are the first 10 unique label values in your data:  ['no' 'yes']
AutoGluon infers your prediction problem is: binary  (because only two unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: [('binary', 'multiclass', 'regression')])

Selected class <--> label mapping:  class 1 = yes, class 0 = no
Feature Generator processed 27800 data points with 17 features
Original Features:
	int features: 6
	float features: 2
	object features: 9
Generated Features:
	int features: 0
All Features:
	int features: 6
	float features: 2
	object features: 9
	Data preprocessing and feature engineering runtime = 0.23s ...
AutoGluon will 

*** Summary of fit() ***
Estimated performance of each model:
                            model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer
0  NeuralNetClassifier_STACKER_l1   0.754524       7.971152  1854.410121                4.870188        1227.118749            1       True
1         weighted_ensemble_k0_l2   0.754524       7.983101  1856.744869                0.011949           2.334748            2       True
2         weighted_ensemble_k0_l1   0.737241       3.113242   629.627097                0.012279           2.335726            1       True
3   LightGBMClassifier_STACKER_l0   0.736925       0.326845    23.771414                0.326845          23.771414            0       True
4   LightGBMClassifier_STACKER_l1   0.733076       3.613902   667.476120                0.512939          40.184749            1       True
5  NeuralNetClassifier_STACKER_l0   0.722692       2.774118   603.519957                2.774118  

In [0]:
doctest = task.Dataset(pd.read_csv('doctor_test.csv'))
y_pred = predictor.predict(doctest)
output = pd.DataFrame({'id': doctest.ID,
                       'Y': y_pred})
output.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')